# Imports

In [116]:
import os
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten
import json

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV



# Feature Extractor

In [92]:
class dataGenerator:
    def __init__(self,basePath,subPath) -> None:
        self.basePath = basePath
        self.subPath = subPath
        self.subset = []
        print(self.basePath,self.subPath)

        self.listAppend()

    def listAppend(self):
        classes = os.listdir(self.basePath+self.subPath)
        print(classes)

        classCounter = 0

        for i in classes:
            imagePaths = os.listdir(self.basePath+self.subPath+f'/{i}/fullsized')
            temp = []
            for image in imagePaths:
                temp.append([f"{classCounter}",self.basePath+self.subPath+ f'/{i}'  '/fullsized' +  f'/{image}'])
                # print(image)
            classCounter+=1
            self.subset += temp
    
    def shuffler(self):
        np.random.seed(0)
        npsubSet = np.array(self.subset)
        np.random.shuffle(npsubSet)
        return npsubSet

In [95]:
# os.remove('All_200X/val/.DS_Store')
os.remove('All_200X/train/.DS_Store')
# os.remove('All_200X/test/.DS_Store')

In [96]:
traindatagen = dataGenerator('All_200X','/train')
trainData = traindatagen.shuffler()

testdatagen = dataGenerator('All_200X','/test')
testData = testdatagen.shuffler()

valdatagen = dataGenerator('All_200X','/val')
valData = valdatagen.shuffler()

All_200X /train
['malignant', 'benign']
All_200X /test
['malignant', 'benign']
All_200X /val
['malignant', 'benign']


In [98]:
class FeatureExtractor:
    def __init__(self,dataset) -> None:
        self.base_model = tf.keras.applications.VGG19(input_shape=(150,150,3),
                                               include_top=False,
                                               weights='imagenet')
        # print(dataset)
        self.dataset = dataset       
        self.extractedFeatures = []    
        self.imagePaths = []
        self.labels = []

        for i in dataset:
            self.labels.append(int(i[0]))
            self.imagePaths.append(i[1])
        
        self.featureCreator()


    def featureCreator(self):
        for i in self.imagePaths:
            
            slicedImages = self.slicer(i)

            currentExtractedFeatures = []

            for image in slicedImages:
                currentExtractedFeatures.append(self.featureExtractor(image).numpy())

            self.extractedFeatures.append(currentExtractedFeatures)



    def slicer(self,imagePath):
        img = plt.imread(imagePath)
        h, w = img.shape[:2]
        ls = []
        for i in range(3):
            roi3 = img[:, int(w*i/3): int(w/3) + int(w*i/3)]
            # plt.imshow(roi)
            for j in range(3):
                roi9 = roi3[int(h*j/3): int(h/3) + int(h*j/3), :]
                ls.append(roi9)
        return ls                    
                                                       
    def preProcessor(self,img):
        img = cv2.resize(img,(150,150))
        imgx = np.expand_dims(img, axis=0)
        return imgx                                        
    
    def featureExtractor(self,image):
        feature = self.base_model(self.preProcessor(image))
        flattened = Flatten()(feature)
        # print(feature.shape,flattened.shape)
        return flattened

In [99]:
valFeatures = FeatureExtractor(valData)

In [100]:
trainFeatures = FeatureExtractor(trainData)

In [101]:
testFeatures = FeatureExtractor(testData)

# SVM Training

In [184]:
def featureScaler(feature):

    maxVal = max(feature)
    minVal = min(feature)

    dezmin = 0
    dezmax = 1

    scaledFeatures =  []
    for i in feature:
        scaled = (    (   (i - minVal)/(maxVal-minVal)  ) /   (dezmax-dezmin)    ) + dezmin
        scaledFeatures.append(scaled)

    return np.array(scaledFeatures)

In [185]:
def SVMDataPreProc(subset):
    subset
    slicedDataset = []
    for j in range(9):
            temp = []
            for i in subset:
                temp.append(featureScaler(i[j][0]))

            slicedDataset.append(temp)
    return slicedDataset

In [193]:
SVMValData = SVMDataPreProc(valFeatures.extractedFeatures)
SVMTrainData = SVMDataPreProc(trainFeatures.extractedFeatures)
SVMTestData = SVMDataPreProc(testFeatures.extractedFeatures)

In [204]:
class multiSVM:
    def __init__(self,Xtrain,yTrain,Xtest,yTest) -> None:
        self.clfs = []
        
        self.Xtrain = Xtrain
        self.yTrain = yTrain
        self.Xtest = Xtest
        self.yTest = yTest


        self.scores = []

    def modelCreator(self):
        for i in range(9):
            self.clfs.append(SVC(kernel='poly', C=6).fit(self.Xtrain[i],self.yTrain))

    def estimator(self):
        for i in range(9):
            y_preds = self.clfs[i].predict(self.Xtest[i])
            self.scores.append(accuracy_score(self.yTest,y_preds))

In [205]:
ensembleSVM = multiSVM(SVMTrainData,trainFeatures.labels,SVMTestData,testFeatures.labels)

In [206]:
ensembleSVM.modelCreator()

In [207]:
ensembleSVM.estimator()
print(ensembleSVM.scores)

[0.8481848184818482, 0.8415841584158416, 0.8382838283828383, 0.834983498349835, 0.8085808580858086, 0.8217821782178217, 0.8415841584158416, 0.834983498349835, 0.8448844884488449]


In [217]:
class MultiSVMPredictor:
    def __init__(self,X,y,classifiers) -> None:
        self.X = X
        self.y = y
        self.classifiers = classifiers
        self.preds = []

        self.prediction()
    
    def prediction(self):
        for i in range(9):
            self.preds.append(self.classifiers[i].predict(self.X[i]))

In [218]:
mpred = MultiSVMPredictor(SVMTestData,testFeatures.labels,ensembleSVM.clfs)

In [221]:
def hardVoting(Preds,Actual):
    for i in range(9):
        for j in Preds:
            print(j[i])